In [1]:
import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model

Using TensorFlow backend.


In [12]:
import pandas as pd

In [13]:
balanced_data=pd.read_csv('balanced.csv')
balanced_data=balanced_data[['Category','Description']]
balanced_data


,Category,Description
0,Upper Body Wear Tops,Cheer on your Florida State Seminoles in this ...
1,Upper Body Wear Tops,Celebrate your Baltimore Ravens fandom during ...
2,Upper Body Wear Tops,A fau sheepskin bat sleeve Dolman top in a cre...
3,Upper Body Wear Tops,Dont let a cold snap keep you from supporting ...
4,Upper Body Wear Tops,The time has come for the Atlantic 10 Conferen...
5,Upper Body Wear Tops,Celebrate your fandom with this Miami Hurrican...
6,Upper Body Wear Tops,Cheer on your Rutgers Scarlet Knights in this ...
7,Upper Body Wear Tops,Celebrate your Florida Gators fandom with this...
8,Upper Body Wear Tops,Celebrate your Maryland Terrapins fandom with ...
9,Upper Body Wear Tops,When the season changes the nights get longer ...


In [27]:
MAX_SEQUENCE_LENGTH =128

EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [15]:
texts=balanced_data['Description']
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences)
labels =balanced_data['Category']
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Found 1143 unique tokens.
Shape of data tensor: (180, 128)
Shape of label tensor: (180,)


In [18]:


# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

In [19]:
embeddings_index = {}
f = open( 'glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [21]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [22]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [30]:
embedding_matrix.shape

(1144, 100)

In [31]:
embed_dim = 100
lstm_out = 30
batch_size= 1

In [32]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import json
import pandas as pd

In [ ]:
model = Sequential()
model.add(Embedding(128, embed_dim,input_length = data.shape[1], dropout=0.1))
model.add(LSTM(lstm_out, dropout_U=0.1, dropout_W=0.1))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

Y = pd.get_dummies(data['sentiment']).values
X_train, X_valid, Y_train, Y_valid = train_test_split(X,Y, test_size = 0.20, random_state = 36)

#Here we train the Network.

model.fit(X_train, Y_train, batch_size =batch_size, nb_epoch = 1,  verbose = 5)

# Measuring score and accuracy on validation set

score,acc = model.evaluate(X_valid, Y_valid, verbose = 2, batch_size = batch_size)
print("Logloss score: %.2f" % (score))
print("Validation set Accuracy: %.2f" % (acc))